In [11]:
# pip install gspread google-auth
# pip install unidecode
# pip install supabase

# Scraped Data Clean-Up

In [23]:
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
import numpy as np

# ------------------------------------------------------------
# 1. Define the OAuth scopes required to access Google Sheets
#    and Google Drive (needed for reading the spreadsheet).
# ------------------------------------------------------------
SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# ------------------------------------------------------------
# 2. Load Google Cloud service account credentials.
#    The JSON file must be downloaded from Google Cloud Console.
#    This file authenticates the script without user interaction.
# ------------------------------------------------------------
creds = Credentials.from_service_account_file(
    "/Users/gomkrtchyan/Downloads/githubanalysis-422613-121df9460e3e.json",
    scopes=SCOPES
)

# ------------------------------------------------------------
# 3. Authorize the client using gspread.
#    This creates a session that can read/write Google Sheets.
# ------------------------------------------------------------
client = gspread.authorize(creds)

# ------------------------------------------------------------
# 4. Open the target spreadsheet by name.
#    Then select the specific worksheet/tab to load.
# ------------------------------------------------------------
SPREADSHEET_NAME = "TripAdvisor Data"
worksheet = client.open(SPREADSHEET_NAME).worksheet("Full Data")

# ------------------------------------------------------------
# 5. Read the entire worksheet into Python.
#    `get_all_records()` returns a list of dictionaries where
#    each dictionary corresponds to a row (header → value).
# ------------------------------------------------------------
records = worksheet.get_all_records()

# ------------------------------------------------------------
# 6. Convert the records into a pandas DataFrame.
#    Pandas provides powerful tools for cleaning, analysis,
#    and preparing the dataset for modeling or a database.
# ------------------------------------------------------------
df = pd.DataFrame(records)

# ------------------------------------------------------------
# 7. Print summary information for quick validation.
# ------------------------------------------------------------
print("Rows loaded:", len(df))
print("Columns:", len(df.columns))

# Display the first few rows to ensure data loaded correctly.
df.head()

Rows loaded: 30835
Columns: 25


,Location Type,Province,Town/City,Location Name,Location Rating,Location # of Reviews,Location Tag,Location Price Range,User Name,User From,...,Restaurant Value Rating,Restaurant Service Rating,Restaurant Food Rating,Restaurant Atmosphere Rating,Hotel Value Rating,Hotel Rooms Rating,Hotel Location Rating,Hotel Cleanliness Rating,Hotel Service Rating,Hotel Sleep Quality Rating
0,Hotel,Shirak,Gyumri,Alexandrapol Hotel,3.7,40,#6 of 10 small hotels in Gyumri,High Cost,Ezia M,,...,,,,,,,,,,
1,Hotel,Shirak,Gyumri,Alexandrapol Hotel,3.7,40,#6 of 10 small hotels in Gyumri,High Cost,Gohar S,,...,,,,,,,,,,
2,Hotel,Shirak,Gyumri,Alexandrapol Hotel,3.7,40,#6 of 10 small hotels in Gyumri,High Cost,Furkat K,,...,,,,,4,5,5,4,5,5
3,Hotel,Shirak,Gyumri,Alexandrapol Hotel,3.7,40,#6 of 10 small hotels in Gyumri,High Cost,Charles_and_Susan,"Nottingham, United Kingdom",...,,,,,,,,,,
4,Hotel,Shirak,Gyumri,Alexandrapol Hotel,3.7,40,#6 of 10 small hotels in Gyumri,High Cost,Jim M,"Dallas, Texas",...,,,,,,,,,,


In [24]:
import re
import pandas as pd
from unidecode import unidecode


# ============================================================
#   EMOJI REMOVAL (no external dependencies)
# ============================================================

def remove_emojis(text):
    """
    Removes emojis using Unicode ranges.
    Works without requiring the `emoji` library.
    """
    if pd.isna(text):
        return ""
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F300-\U0001F5FF"  # Symbols & pictographs
        "\U0001F680-\U0001F6FF"  # Transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # Flags
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "\U0001F900-\U0001F9FF"  # Supplemental symbols
        "\U0001FA70-\U0001FAFF"  # Extended symbols
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub("", str(text))


# ============================================================
#   TRANSLITERATION FOR ARMENIAN / RUSSIAN / PERSIAN / ARABIC
# ============================================================
def transliterate_text(text):
    """
    Converts Armenian, Russian, Persian, Arabic, etc. characters
    into Latin equivalents using `unidecode`.
    Also strips invisible directional markers.
    """
    if pd.isna(text):
        return ""
    s = unidecode(str(text))
    s = re.sub(r"[\u200B-\u200F\u202A-\u202E]", "", s)  # remove RTL + zero-width
    return s


# ============================================================
#   GENERIC CLEANER: emojis → transliteration → ASCII-safe
# ============================================================
def clean_text(text):
    """
    Full sanitation pipeline:
    - Remove emojis
    - Transliterate foreign scripts
    - Strip non-ASCII
    - Normalize whitespace
    """
    if pd.isna(text):
        return ""

    s = str(text)
    s = remove_emojis(s)
    s = transliterate_text(s)

    # Remove non-ASCII after transliteration
    s = re.sub(r"[^\x00-\x7F]+", " ", s)

    # Collapse multiple spaces
    s = re.sub(r"\s{2,}", " ", s).strip()
    return s


# ============================================================
#   CLEAN USERNAME (ASCII-safe)
# ============================================================
def clean_username(text):
    return clean_text(text)


# ============================================================
#   CLEAN REVIEW SUBJECT (ASCII-safe + quoted)
# ============================================================
def clean_subject(text):
    cleaned = clean_text(text)
    return f"\"{cleaned}\""


# ============================================================
#   CLEAN REVIEW TEXT (ASCII-safe + quoted)
# ============================================================
def clean_review_text(text):
    cleaned = clean_text(text)
    return f"\"{cleaned}\""


# ============================================================
#   PRICE RANGE NORMALIZATION
# ============================================================
def normalize_price(val):
    """
    Normalizes TripAdvisor-style price ranges:
    - "$"            → Low Cost
    - "$$ - $$$"     → Medium Cost
    - "$$$$"         → High Cost
    If no $ appears in the value, return the value unchanged.
    """
    if pd.isna(val):
        return ""

    s = str(val).strip()

    # If there are no dollar signs at all → return original
    if "$" not in s:
        return s

    mapping = {
        "$": "Low Cost",
        "$ - $$$": "Low Cost",
        "$-$$$": "Low Cost",

        "$$ - $$$": "Medium Cost",
        "$$-$$$": "Medium Cost",

        "$$$$": "High Cost"
    }

    # Direct mapping
    if s in mapping:
        return mapping[s]

    # Fallback: count number of $
    count = s.count("$")
    if count == 1:
        return "Low Cost"
    elif count == 2:
        return "Medium Cost"
    elif count >= 3:
        return "High Cost"

    return s


# ============================================================
#   EXTRACT FIRST NUMBER FROM "X of Y bubbles"
# ============================================================
def extract_first_number(val):
    """
    Extracts the leading number from:
    - "4 of 5 bubbles"
    - "3 bubbles"
    - "5"
    Returns blank if NaN.
    """
    if pd.isna(val):
        return ""

    match = re.match(r"\s*(\d+)", str(val))
    return match.group(1) if match else ""


# ============================================================
#   FIX BLANK TYPE COLUMN
# ============================================================
def clean_type(val):
    """
    Sets missing values in column N to "None".
    """
    if pd.isna(val) or str(val).strip() == "":
        return "None"
    return clean_text(val)


# ============================================================
#   APPLY CLEANING PIPELINE TO DATAFRAME
# ============================================================
def clean_dataframe(df):
    """
    Cleans the full TripAdvisor dataset using the defined rules.
    """

    # ----- Username (Column I) -----
    df["User Name"] = df["User Name"].apply(clean_username)

    # ----- Review Subject (Column L) -----
    df["Review Subject"] = df["Review Subject"].apply(clean_subject)

    # ----- Review Text (Column O) -----
    df["Review Text"] = df["Review Text"].apply(clean_review_text)

    # ----- Price Range (Column H) -----
    df["Location Price Range"] = df["Location Price Range"].apply(normalize_price)

    # ----- Ratings (K, P → Y) -----
    rating_columns = [
        "Review Rating",
        "Restaurant Value Rating",
        "Restaurant Service Rating",
        "Restaurant Food Rating",
        "Restaurant Atmosphere Rating",
        "Hotel Value Rating",
        "Hotel Rooms Rating",
        "Hotel Location Rating",
        "Hotel Cleanliness Rating",
        "Hotel Service Rating",
        "Hotel Sleep Quality Rating"
    ]

    for col in rating_columns:
        df[col] = df[col].apply(extract_first_number)

    # ----- Fix blank Type column (Column N) -----
    df["Type"] = df["Type"].apply(clean_type)

    return df


In [25]:
from IPython.display import display, HTML

html = df.to_html(max_rows=500)
display(HTML(f"<div style='height:400px; overflow-y:scroll;'>{html}</div>"))

,Location Type,Province,Town/City,Location Name,Location Rating,Location # of Reviews,Location Tag,Location Price Range,User Name,User From,Review Rating,Review Subject,Review Date,Type,Review Text,Restaurant Value Rating,Restaurant Service Rating,Restaurant Food Rating,Restaurant Atmosphere Rating,Hotel Value Rating,Hotel Rooms Rating,Hotel Location Rating,Hotel Cleanliness Rating,Hotel Service Rating,Hotel Sleep Quality Rating
0,Hotel,Shirak,Gyumri,Alexandrapol Hotel,3.7,40,#6 of 10 small hotels in Gyumri,High Cost,Ezia M,,1,Attention when booking,2025-09-01,Friends,Upon arrival we are informed that unfortunately the room is not available due to inefficiencies in the bathrooms of the 3 available rooms. They book us in a lower category hotel 3 km outside the center. Obviously the disruption was an excuse not to give us the room we had booked at a very low price the day before through booking. You see if you want to trust,,,,,,,,,,
1,Hotel,Shirak,Gyumri,Alexandrapol Hotel,3.7,40,#6 of 10 small hotels in Gyumri,High Cost,Gohar S,,1,The terrible hotel in the world,2025-06-01,Family,"We book hotel for 50 people, for the class and their parents and pay for it before. After arriving a real chaos is starting when finding that mostt of rooms we have reserved are have more important guests alreasy. This was a terrible situatiin that I have ever met. Only a young girl like administrator was in place. NO ONE IS WORRY ABOUT THIS. I will prefer to sleep in street rather chose thi hotel. Thonk about it before reserbing.",,,,,,,,,,
2,Hotel,Shirak,Gyumri,Alexandrapol Hotel,3.7,40,#6 of 10 small hotels in Gyumri,High Cost,Furkat K,,5,I really liked the calving super I was March 24 days left. I have no complaint. Breakfast is great. Especially the employees.,2024-03-01,Solo,The room is clean and quiet 🔕. Everything was in the room. I hope to fly there again for a long time. Breakfast is also nothing 👍The employees are polite people 👯.The price is also not a big deal. Okay. Class class everything is super 👍 👍 #👍## 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍,,,,,4,5,5,4,5,5
3,Hotel,Shirak,Gyumri,Alexandrapol Hotel,3.7,40,#6 of 10 small hotels in Gyumri,High Cost,Charles_and_Susan,"Nottingham, United Kingdom",4,"Comfortable, central and Wow!",2022-09-01,None,"Our itinerary in Armenia was changed to avoid the troubled areas. We weren't supposed to be staying in Gyumri so this came as a surprise. It's like walking into a Disneyland Hotel, totally over the top decoration but it works fine.The rooms follow a similar decorated pattern, the facilities are good. We had our dinner at the hotel which was good, the breakfast fine too. It's fairly central - opposite one of the main squares and near to the two pedestion shopping streets.",,,,,,,,,,
4,Hotel,Shirak,Gyumri,Alexandrapol Hotel,3.7,40,#6 of 10 small hotels in Gyumri,High Cost,Jim M,"Dallas, Texas",4,Very ornate hotel in a good location,2019-09-01,Solo,"This is a ""over the top"" ornate hotel within a five minute walking distance ti thecmain square and local markets, restaurants and shops. The rooms and main areas are elaborately decorated with marble staircases, statutes and fabric.",,,,,,,,,,
5,Hotel,Shirak,Gyumri,Alexandrapol Hotel,3.7,40,#6 of 10 small hotels in Gyumri,High Cost,N.T.,"Yerevan, Armenia",3,overdose,2018-11-01,Business,"nice hotel fir overnight stay, not more... luxury decoration and settings are too much and impressionbis that you are staying inside of the museum. Facilities are good, shower cabine, some toiletories, wifi. Bed is very inconvenient and problems with heating.",,,,,,,,,,
6,Hotel,Shirak,Gyumri,Alexandrapol Hotel,3.7,40,#6 of 10 small hotels in Gyumri,High Cost,Jan V,"Antwerp, Belgium",5,extraordinary experience,2018-08-01,Solo,I only went in to have coffee in the lounge. I saw no guests but the the staff was very friendly. The setting is extraordinary as you lounge in the voluptuous leather sofa's or armchairs. Good coffee as well.,,,,,5,5,5,5,5,5
7,Hotel,Shirak,Gyumri,Alexand

# Table Creation & ID Assignment

In [26]:
import pandas as pd

# ==========================================================================================
#   BUILD RELATIONAL TABLES FOR SUPABASE
#   Schema:
#       geolocation
#       locations
#       app_users
#       reviews
#       restaurant_review_details
#       hotel_review_details
#
#   Input:
#       df  → The fully cleaned DataFrame loaded from Google Sheets.
#
#   Output:
#       Six relational DataFrames, each ready for Supabase insertion.
# ==========================================================================================


def build_relational_tables(df):
    """
    Given the cleaned TripAdvisor dataframe, generate normalized relational tables
    following the Supabase schema.

    This function:
        - Deduplicates geolocations
        - Assigns location IDs
        - Deduplicates users
        - Assigns review IDs
        - Splits restaurant vs hotel details
        - Preserves foreign key relationships

    Returns:
        geolocation_df
        locations_df
        users_df
        reviews_df
        restaurant_details_df
        hotel_details_df
    """

    # ======================================================================================
    # 1. -------------------------   BUILD GEOLOCATION TABLE   -----------------------------
    # ======================================================================================
    #
    # Geolocation columns come directly from the sheet:
    #   Province, Town/City
    #
    # Each unique pair gets a unique geolocation_id.
    #

    geo_cols = ["Province", "Town/City"]
    geolocation_df = (
        df[geo_cols]
        .drop_duplicates()
        .reset_index(drop=True)
        .rename(columns={
            "Province": "province",
            "Town/City": "town_city"
        })
        .copy()
)

    geolocation_df["geolocation_id"] = geolocation_df.index + 1000001  # stable surrogate keys

    # Create lookup dict for assigning FK later
    geo_lookup = {
        (row["province"], row["town_city"]): row["geolocation_id"]
        for _, row in geolocation_df.iterrows()
}



    # ======================================================================================
    # 2. ----------------------------   BUILD LOCATIONS TABLE   -----------------------------
    # ======================================================================================
    #
    # Columns:
    #   location_id (PK)
    #   geolocation_id (FK)
    #   location_type
    #   location_name
    #   location_rating
    #   location_num_reviews
    #   location_tag
    #   location_price_range
    #

    loc_cols = [
        "Location Type",
        "Location Name",
        "Location Rating",
        "Location # of Reviews",
        "Location Tag",
        "Location Price Range",
        "Province",
        "Town/City",
    ]

    locations_df = (
        df[loc_cols]
        .drop_duplicates()
        .reset_index(drop=True)
        .copy()
    )

    # Assign geolocation FK
    locations_df["geolocation_id"] = locations_df.apply(
        lambda r: geo_lookup[(r["Province"], r["Town/City"])],
        axis=1
    )

    # Assign location_id
    locations_df["location_id"] = locations_df.index + 2000001

    # Prepare final table with correct column order
    locations_df = locations_df.rename(columns={
        "Location Type": "location_type",
        "Location Name": "location_name",
        "Location Rating": "location_rating",
        "Location # of Reviews": "location_num_reviews",
        "Location Tag": "location_tag",
        "Location Price Range": "location_price_range",
    })

    locations_df = locations_df[[
        "location_id",
        "geolocation_id",
        "location_type",
        "location_name",
        "location_rating",
        "location_num_reviews",
        "location_tag",
        "location_price_range"
    ]]


    # Create lookup for FK mapping (location_name + geo → location_id)
    loc_lookup = {
        (row["location_name"], row["geolocation_id"]): row["location_id"]
        for _, row in locations_df.iterrows()
    }


    # ======================================================================================
    # 3. -------------------------------   BUILD USERS TABLE   ------------------------------
    # ======================================================================================
    #
    # Columns:
    #   user_id (PK)
    #   user_name
    #   user_from
    #

    users_cols = ["User Name", "User From"]

    users_df = (
        df[users_cols]
        .drop_duplicates()
        .reset_index(drop=True)
        .copy()
    )

    # Assign user_id
    users_df["user_id"] = users_df.index + 3000001

    # Rename for Supabase schema
    users_df = users_df.rename(columns={
        "User Name": "user_name",
        "User From": "user_from"
    })

    # Create lookup for FK assignment
    user_lookup = {
        (row["user_name"], row["user_from"]): row["user_id"]
        for _, row in users_df.iterrows()
    }


    # ======================================================================================
    # 4. -------------------------------   BUILD REVIEWS TABLE   ----------------------------
    # ======================================================================================
    #
    # Columns:
    #   review_id (PK)
    #   location_id (FK)
    #   user_id (FK)
    #   review_rating
    #   review_subject
    #   review_text
    #   review_date
    #   review_type
    #

    review_cols = [
        "Location Name",
        "Province",
        "Town/City",
        "User Name",
        "User From",
        "Review Rating",
        "Review Subject",
        "Review Text",
        "Review Date",
        "Type",
    ]

    reviews_df = df[review_cols].copy().reset_index(drop=True)

    # Assign location FK
    reviews_df["geolocation_id"] = reviews_df.apply(
        lambda r: geo_lookup[(r["Province"], r["Town/City"])],
        axis=1
    )

    reviews_df["location_id"] = reviews_df.apply(
        lambda r: loc_lookup[(r["Location Name"], r["geolocation_id"])],
        axis=1
    )

    # Assign user FK
    reviews_df["user_id"] = reviews_df.apply(
        lambda r: user_lookup[(r["User Name"], r["User From"])],
        axis=1
    )

    # Assign review_id
    reviews_df["review_id"] = reviews_df.index + 5000001

    # Rename according to schema
    reviews_df = reviews_df.rename(columns={
        "Review Rating": "review_rating",
        "Review Subject": "review_subject",
        "Review Text": "review_text",
        "Review Date": "review_date",
        "Type": "review_type"
    })

    # Final column order
    reviews_df = reviews_df[[
        "review_id",
        "location_id",
        "user_id",
        "review_rating",
        "review_subject",
        "review_text",
        "review_date",
        "review_type"
    ]]


    # ======================================================================================
    # 5. ---------------------   RESTAURANT REVIEW DETAILS TABLE   -------------------------
    # ======================================================================================

    rest_cols = [
        "Restaurant Value Rating",
        "Restaurant Service Rating",
        "Restaurant Food Rating",
        "Restaurant Atmosphere Rating"
    ]

    restaurant_details_df = df[rest_cols].copy()
    restaurant_details_df["review_id"] = reviews_df["review_id"]

    restaurant_details_df = restaurant_details_df.rename(columns={
        "Restaurant Value Rating": "value_rating",
        "Restaurant Service Rating": "service_rating",
        "Restaurant Food Rating": "food_rating",
        "Restaurant Atmosphere Rating": "atmosphere_rating"
    })

    restaurant_details_df = restaurant_details_df[
        ["review_id", "value_rating", "service_rating", "food_rating", "atmosphere_rating"]
    ]


    # ======================================================================================
    # 6. ------------------------   HOTEL REVIEW DETAILS TABLE   ---------------------------
    # ======================================================================================

    hotel_cols = [
        "Hotel Value Rating",
        "Hotel Rooms Rating",
        "Hotel Location Rating",
        "Hotel Cleanliness Rating",
        "Hotel Service Rating",
        "Hotel Sleep Quality Rating"
    ]

    hotel_details_df = df[hotel_cols].copy()
    hotel_details_df["review_id"] = reviews_df["review_id"]

    hotel_details_df = hotel_details_df.rename(columns={
        "Hotel Value Rating": "value_rating",
        "Hotel Rooms Rating": "rooms_rating",
        "Hotel Location Rating": "location_rating",
        "Hotel Cleanliness Rating": "cleanliness_rating",
        "Hotel Service Rating": "service_rating",
        "Hotel Sleep Quality Rating": "sleep_quality_rating"
    })

    hotel_details_df = hotel_details_df[
        ["review_id", "value_rating", "rooms_rating", "location_rating",
         "cleanliness_rating", "service_rating", "sleep_quality_rating"]
    ]

    # ======================================================================================
    #  RETURN ALL TABLES
    # ======================================================================================
    return (
        geolocation_df,
        locations_df,
        users_df,
        reviews_df,
        restaurant_details_df,
        hotel_details_df
    )

In [27]:
geo_df, loc_df, users_df, reviews_df, rest_df, hotel_df = build_relational_tables(df)

# DB Connection & Bulk Insert

In [28]:
from supabase import create_client, Client
import pandas as pd
import math
import time

# ==============================================================================
#  SUPABASE CONNECTION
# ==============================================================================

SUPABASE_URL = "https://kqyaqgsrcahvtesoimsh.supabase.co"
SUPABASE_KEY = "sb_secret_G6kFTi07P63ysXCMJC8gLw_nlV6_pxF"

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

In [29]:
# ==============================================================================
#  HELPER — INSERT IN BATCHES (recommended for large datasets)
# ==============================================================================

def insert_in_batches(table_name, df, batch_size=500):
    """
    Upserts rows into Supabase in batches.
    Ensures JSON-compliant values by converting:
    - NaN -> None
    - empty strings "" -> None
    """
    total_rows = len(df)
    print(f"Upserting into '{table_name}' ({total_rows} rows)...")

    for start in range(0, total_rows, batch_size):
        end = min(start + batch_size, total_rows)

        batch_df = df.iloc[start:end].copy()

        # 1. Replace NaN → None
        batch_df = batch_df.where(pd.notnull(batch_df), None)

        # 2. Replace empty string "" → None (per column)
        batch_df = batch_df.apply(lambda col: col.map(lambda x: None if x == "" else x))

        # 3. Convert to records AFTER sanitizing
        batch = batch_df.to_dict(orient="records")

        # 4. Upsert into Supabase
        response = supabase.table(table_name).upsert(batch).execute()

        dump = response.model_dump() if hasattr(response, "model_dump") else response

        if dump.get("status_code", 200) >= 400:
            print(f"❌ Error upserting rows {start}-{end}: {dump}")
            raise Exception(dump)

        print(f"✅ Upserted rows {start}-{end}")

    print(f"🎉 Finished upserting into '{table_name}'\n")

In [36]:
# ==============================================================================
#  MAIN LOAD FUNCTION
# ==============================================================================

def load_all_tables_into_supabase(geo_df, locations_df, users_df,
                                  reviews_df, restaurant_details_df,
                                  hotel_details_df):
    """
    Inserts all relational tables into Supabase in correct order.
    This avoids def insert_in_batches(table_name, df, batch_size=500):
    total_rows = len(df)
    print(f"Inserting/upserting into '{table_name}' ({total_rows} rows)...")

    for start in range(0, total_rows, batch_size):
        end = min(start + batch_size, total_rows)

        batch_df = df.iloc[start:end].where(pd.notnull(df.iloc[start:end]), None)
        batch = batch_df.to_dict(orient="records")

        response = supabase.table(table_name).upsert(batch).execute()

        dump = response.model_dump() if hasattr(response, "model_dump") else response

        if dump.get("status_code", 200) >= 400:
            print(f"❌ Error inserting batch {start}-{end}: {dump}")
            raise Exception(dump)

        print(f"✅ Upserted rows {start}-{end}")

    print(f"🎉 Completed upserts for '{table_name}'\n")
foreign key constraint failures.
    """

    # 1️⃣ Insert geolocation table
    insert_in_batches("geolocation", geo_df)

    # 2️⃣ Insert locations table (depends on geolocation)
    insert_in_batches("locations", locations_df)

    # 3️⃣ Insert app_users table
    insert_in_batches("app_users", users_df)

    # 4️⃣ Insert reviews table (depends on locations + users)
    insert_in_batches("reviews", reviews_df)

    # 5️⃣ Insert restaurant details (depends on reviews)
    insert_in_batches("restaurant_review_details", restaurant_details_df)

    # 6️⃣ Insert hotel details (depends on reviews)
    insert_in_batches("hotel_review_details", hotel_details_df)

    print("\n🎉 ALL TABLES SUCCESSFULLY INSERTED INTO SUPABASE\n")

def make_json_safe(df):
    """
    Ensures dataframe is JSON-safe:
    - Replaces NaN/NaT/pandas NA with None
    - Replaces "" with None
    """
    # Replace all NaN types → None
    df = df.replace({np.nan: None, pd.NA: None, pd.NaT: None})
    
    # Replace empty strings → None
    df = df.apply(lambda col: col.map(lambda x: None if x == "" else x))

    return df

# Apply JSON sanitation to every table
geo_df     = make_json_safe(geo_df)
loc_df     = make_json_safe(loc_df)
users_df   = make_json_safe(users_df)
reviews_df = make_json_safe(reviews_df)
rest_df    = make_json_safe(rest_df)
hotel_df   = make_json_safe(hotel_df)

def fix_nan(df):
    return df.replace({
        np.nan: None,
        pd.NA: None,
        pd.NaT: None
    }).astype(object)

geo_df     = fix_nan(geo_df)
loc_df     = fix_nan(loc_df)
users_df   = fix_nan(users_df)
reviews_df = fix_nan(reviews_df)
rest_df    = fix_nan(rest_df)
hotel_df   = fix_nan(hotel_df)

def stringify_df(df):
    """
    Converts ALL values in a dataframe to strings so JSON encoding NEVER fails.
    Converts NaN to None-like strings (“”) for cleaner output.
    Ensures all values fit VARCHAR types in Supabase.
    """
    return df.astype(str).replace({"nan": None})

# Convert BOTH detail tables to string-safe format
rest_df  = stringify_df(rest_df)
hotel_df = stringify_df(hotel_df)

In [37]:
load_all_tables_into_supabase(
    geo_df,
    loc_df,
    users_df,
    reviews_df,
    rest_df,
    hotel_df
)

Upserting into 'geolocation' (40 rows)...
✅ Upserted rows 0-40
🎉 Finished upserting into 'geolocation'

Upserting into 'locations' (360 rows)...
✅ Upserted rows 0-360
🎉 Finished upserting into 'locations'

Upserting into 'app_users' (22465 rows)...
✅ Upserted rows 0-500
✅ Upserted rows 500-1000
✅ Upserted rows 1000-1500
✅ Upserted rows 1500-2000
✅ Upserted rows 2000-2500
✅ Upserted rows 2500-3000
✅ Upserted rows 3000-3500
✅ Upserted rows 3500-4000
✅ Upserted rows 4000-4500
✅ Upserted rows 4500-5000
✅ Upserted rows 5000-5500
✅ Upserted rows 5500-6000
✅ Upserted rows 6000-6500
✅ Upserted rows 6500-7000
✅ Upserted rows 7000-7500
✅ Upserted rows 7500-8000
✅ Upserted rows 8000-8500
✅ Upserted rows 8500-9000
✅ Upserted rows 9000-9500
✅ Upserted rows 9500-10000
✅ Upserted rows 10000-10500
✅ Upserted rows 10500-11000
✅ Upserted rows 11000-11500
✅ Upserted rows 11500-12000
✅ Upserted rows 12000-12500
✅ Upserted rows 12500-13000
✅ Upserted rows 13000-13500
✅ Upserted rows 13500-14000
✅ Upserted 